In [44]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns
from scipy.stats import pearsonr

from zipfile import ZipFile
import scipy as sp
import re


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap


from scipy.special import binom

In [45]:
log_file_path = "/Users/mparedes/Desktop/mpox_rhino/monthly_pred/combined_exp_glm.log"


In [62]:
def read_in_Ne_changes_mascot(log_file_path):  
    
    Ne_skyline_dict = {"sample":[]}

    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if ("NeGLM."  in col) or ("migrationGLM." in col):
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "Ne.region.1" where region is deme and 1 is interval 1
                        #interval = name.split(".")[2]

                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])

    return(Ne_skyline_dict)

In [63]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

In [64]:
Ne_skyline

{'sample': ['0',
  '5000',
  '10000',
  '15000',
  '20000',
  '25000',
  '30000',
  '35000',
  '40000',
  '45000',
  '50000',
  '55000',
  '60000',
  '65000',
  '70000',
  '75000',
  '80000',
  '85000',
  '90000',
  '95000',
  '100000',
  '105000',
  '110000',
  '115000',
  '120000',
  '125000',
  '130000',
  '135000',
  '140000',
  '145000',
  '150000',
  '155000',
  '160000',
  '165000',
  '170000',
  '175000',
  '180000',
  '185000',
  '190000',
  '195000',
  '200000',
  '205000',
  '210000',
  '215000',
  '220000',
  '225000',
  '230000',
  '235000',
  '240000',
  '245000',
  '250000',
  '255000',
  '260000',
  '265000',
  '270000',
  '275000',
  '280000',
  '285000',
  '290000',
  '295000',
  '300000',
  '305000',
  '310000',
  '315000',
  '320000',
  '325000',
  '330000',
  '335000',
  '340000',
  '345000',
  '350000',
  '355000',
  '360000',
  '365000',
  '370000',
  '375000',
  '380000',
  '385000',
  '390000',
  '395000',
  '400000',
  '405000',
  '410000',
  '415000',
  '4200

In [69]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        print(i)
        if ("NeGLM."in i) or ("migrationGLM" in i):
            if "Clock" in i: 
                deme = i.split(".")[1]
            else:
                deme = i.split(".")[2]
            #print(deme)
            #interval = i.split(".")[2]
            #print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = 10**mean_log
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = 10**lower_hpd_log_95
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = 10**upper_hpd_log_95
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = 10**lower_hpd_log_50
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = 10**upper_hpd_log_50
            
#             try:
#                 next_local_series = input_df["Ne"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
#                 diff_series = np.subtract(local_series, next_local_series)
#                 #print(local_series)
#                 #print(next_local_series)
#                 #print(diff_series)
#                 diff_mean_log = diff_series.mean()
#                 diff_median_log = np.median(diff_series)
#                 diff_hpd_95 = az.hdi(diff_series, 0.95)
#                 diff_lower_hpd_log_95 = diff_hpd_95[0]
#                 diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
#                 diff_upper_hpd_log_95 = diff_hpd_95[1]
#                 diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
#                 diff_hpd_50 = az.hdi(diff_series, 0.50)
#                 diff_lower_hpd_log_50 = diff_hpd_50[0]
#                 diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
#                 diff_upper_hpd_log_50 = diff_hpd_50[1]
#                 diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
#             except KeyError:
#                 pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95})
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [66]:
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

28124


,sample,migrationGLM._region_300Clock,migrationGLM._region_300scaler.regions_air_travel_matrix,NeGLM._region_300Clock,NeGLM._region_300scaler.region_cases_weekly_prevalence,NeGLM._region_300scaler.month_predictor_0,NeGLM._region_300scaler.month_predictor_1,NeGLM._region_300scaler.month_predictor_2,NeGLM._region_300scaler.month_predictor_3,NeGLM._region_300scaler.month_predictor_4,NeGLM._region_300scaler.month_predictor_5,NeGLM._region_300scaler.month_predictor_6,NeGLM._region_300scaler.month_predictor_7,NeGLM._region_300scaler.month_predictor_8,NeGLM._region_300scaler.month_predictor_9
0,0,0.26283241860558926,1.2480350435234993,1.067550747103468,1.8765474835457785,-1.081670441012801,1.0604562770300139,0.828021719658522,-0.26299329921108305,-0.048047963974110264,0.0,-0.6443972073850182,0.916582374323458,-2.2466374301088843,0.14169323776729104
1,5000,0.1734771530121066,1.53571719193313,1.0551279305224224,1.8765474835457785,-1.081670441012801,1.0604562770300139,0.828021719658522,-0.2768564532159947,-0.03147032298607799,-0.27466274478971964,-1.025595641021285,0.26241240549374956,-0.0028740614266937747,0.0
2,10000,0.24433213400555528,1.3923594894412048,1.6740314395740168,1.6155438293788356,-1.646049591262696,0.5142870385761356,0.41612412923327646,-0.079843151185925,-0.05762473970940296,-0.9179841383249463,-1.4892815308302374,0.6052177356160429,-0.4686729595438821,-2.2312926976726026
3,15000,0.11535924410602917,2.0887571531080957,0.9629094726412906,1.8765474835457785,-0.9037942324740789,0.9052943930506832,0.8357257714717867,-0.07721118593631071,0.0,0.0,-0.6060230039278118,0.2925908230980183,-0.39562992923584694,0.1099572453629114
4,20000,0.09524901543702931,2.1067857249756514,0.39947649077505104,2.1902364598710498,0.0,1.689465114744717,1.3106580388835198,1.0305076429909972,0.0,0.0,-0.42954877935397207,0.0,0.0,0.35615398042599855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28119,140595000,0.19346863273908024,1.258407064727173,1.5612849612836746,1.6706159415183528,-1.8642957338475181,0.4626630523047641,0.16143557302545952,0.2782269503335515,-0.632286826161373,0.8895495754858267,-0.7414199195025191,-0.44714322776578763,-0.7047265251291224,0.4308055655381722
28120,140600000,0.14390571628896195,1.58417286131803,1.4112547432346934,1.818190971710826,-1.6735757535143811,0.4433047256806515,0.16143557302545952,0.0,-1.0262933159660463,0.13090903163465883,-0.8174540041489253,0.009754100791802278,-1.3195482759646582,-0.14326743068416037
28121,140605000,0.13025441371462426,1.5088171496677627,1.6897164073761584,1.818190971710826,-1.8790560511191559,0.4433047256806515,0.0,-0.1673647528231128,-1.0262933159660463,0.0,-1.620165877571604,-0.005442897197203145,-1.2260230774628242,0.10892519044453675
28122,140610000,0.16007302469502172,1.4407196563267037,1.6837865377220484,1.818190971710826,-1.9235924478142445,0.4433047256806515,0.0,-0.1673647528231128,-1.0262933159660463,0.0,-1.6085617446043554,-0.7870108450012607,-1.7406107050693773,-0.048957109162794055


In [67]:
Ne_df

,sample,migrationGLM._region_300Clock,migrationGLM._region_300scaler.regions_air_travel_matrix,NeGLM._region_300Clock,NeGLM._region_300scaler.region_cases_weekly_prevalence,NeGLM._region_300scaler.month_predictor_0,NeGLM._region_300scaler.month_predictor_1,NeGLM._region_300scaler.month_predictor_2,NeGLM._region_300scaler.month_predictor_3,NeGLM._region_300scaler.month_predictor_4,NeGLM._region_300scaler.month_predictor_5,NeGLM._region_300scaler.month_predictor_6,NeGLM._region_300scaler.month_predictor_7,NeGLM._region_300scaler.month_predictor_8,NeGLM._region_300scaler.month_predictor_9
0,0,0.26283241860558926,1.2480350435234993,1.067550747103468,1.8765474835457785,-1.081670441012801,1.0604562770300139,0.828021719658522,-0.26299329921108305,-0.048047963974110264,0.0,-0.6443972073850182,0.916582374323458,-2.2466374301088843,0.14169323776729104
1,5000,0.1734771530121066,1.53571719193313,1.0551279305224224,1.8765474835457785,-1.081670441012801,1.0604562770300139,0.828021719658522,-0.2768564532159947,-0.03147032298607799,-0.27466274478971964,-1.025595641021285,0.26241240549374956,-0.0028740614266937747,0.0
2,10000,0.24433213400555528,1.3923594894412048,1.6740314395740168,1.6155438293788356,-1.646049591262696,0.5142870385761356,0.41612412923327646,-0.079843151185925,-0.05762473970940296,-0.9179841383249463,-1.4892815308302374,0.6052177356160429,-0.4686729595438821,-2.2312926976726026
3,15000,0.11535924410602917,2.0887571531080957,0.9629094726412906,1.8765474835457785,-0.9037942324740789,0.9052943930506832,0.8357257714717867,-0.07721118593631071,0.0,0.0,-0.6060230039278118,0.2925908230980183,-0.39562992923584694,0.1099572453629114
4,20000,0.09524901543702931,2.1067857249756514,0.39947649077505104,2.1902364598710498,0.0,1.689465114744717,1.3106580388835198,1.0305076429909972,0.0,0.0,-0.42954877935397207,0.0,0.0,0.35615398042599855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28119,140595000,0.19346863273908024,1.258407064727173,1.5612849612836746,1.6706159415183528,-1.8642957338475181,0.4626630523047641,0.16143557302545952,0.2782269503335515,-0.632286826161373,0.8895495754858267,-0.7414199195025191,-0.44714322776578763,-0.7047265251291224,0.4308055655381722
28120,140600000,0.14390571628896195,1.58417286131803,1.4112547432346934,1.818190971710826,-1.6735757535143811,0.4433047256806515,0.16143557302545952,0.0,-1.0262933159660463,0.13090903163465883,-0.8174540041489253,0.009754100791802278,-1.3195482759646582,-0.14326743068416037
28121,140605000,0.13025441371462426,1.5088171496677627,1.6897164073761584,1.818190971710826,-1.8790560511191559,0.4433047256806515,0.0,-0.1673647528231128,-1.0262933159660463,0.0,-1.620165877571604,-0.005442897197203145,-1.2260230774628242,0.10892519044453675
28122,140610000,0.16007302469502172,1.4407196563267037,1.6837865377220484,1.818190971710826,-1.9235924478142445,0.4433047256806515,0.0,-0.1673647528231128,-1.0262933159660463,0.0,-1.6085617446043554,-0.7870108450012607,-1.7406107050693773,-0.048957109162794055


In [70]:
ne_summary = generate_summary_df(Ne_df)
ne_summary

sample
migrationGLM._region_300Clock
migrationGLM._region_300scaler.regions_air_travel_matrix
NeGLM._region_300Clock
NeGLM._region_300scaler.region_cases_weekly_prevalence
NeGLM._region_300scaler.month_predictor_0
NeGLM._region_300scaler.month_predictor_1
NeGLM._region_300scaler.month_predictor_2
NeGLM._region_300scaler.month_predictor_3
NeGLM._region_300scaler.month_predictor_4
NeGLM._region_300scaler.month_predictor_5
NeGLM._region_300scaler.month_predictor_6
NeGLM._region_300scaler.month_predictor_7
NeGLM._region_300scaler.month_predictor_8
NeGLM._region_300scaler.month_predictor_9


/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_99919/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_99919/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_99919/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_99919/3821785173.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_

,deme,mean_Ne_log,mean_Ne_linear,median_Ne_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,lower_hpd_linear
0,_region_300Clock,0.189070,1.545505,0.180239,0.309295,0.081095,0.205719,0.131006,2.038424,1.205300
0,regions_air_travel_matrix,1.502342,31.793794,1.503020,2.156718,0.862050,1.740284,1.289982,143.455766,7.278643
0,_region_300Clock,0.974816,9.436609,0.916874,1.741570,0.345521,1.061502,0.605930,55.153080,2.215750
0,region_cases_weekly_prevalence,1.811690,64.817149,1.809091,2.333381,1.293268,1.998874,1.643755,215.467098,19.645716
0,month_predictor_0,-1.090373,0.081213,-1.105467,0.000000,-1.890191,-0.836518,-1.453160,1.000000,0.012877
0,month_predictor_1,1.349364,22.354456,1.327705,2.295658,0.448627,1.591076,0.974446,197.541130,2.809489
0,month_predictor_2,0.879787,7.582063,0.903319,1.605495,0.000000,1.217403,0.630435,40.317611,1.000000
0,month_predictor_3,0.350134,2.239410,0.265768,1.271516,-0.359538,0.447114,-0.001437,18.685979,0.436981
0,month_predictor_4,-0.218935,0.604039,0.000000,0.551484,-1.181620,0.000518,-0.337461,3.560283,0.065823
0,month_predictor_5,-0.279185,0.525793,-0.106759,0.612273,-1.455882,0.000133,-0.474297,4.095176,0.035004


In [94]:
month = {'month_predictor_0':'April 2022',
          'month_predictor_1':'May 2022',
         'month_predictor_2':'June 2022',
           'month_predictor_3': 'July 2022',
        'month_predictor_4':'August 2022', 
        'month_predictor_5':'September 2022',
         'month_predictor_6':'October 2022',
        'month_predictor_7':'November 2022',
        'month_predictor_8':'Decemeber 2022',
        'month_predictor_9':'Janaury 2023',
         'region_cases_weekly':'Weekly Cases',
         'region_cases_weekly_prevalence':'Weekly Prevalence',
         '_region_300Clock':'Clock', 
         "regions_air_travel_matrix":"Air Passenger Volumes"
}

In [95]:
ne_summary["predictor"] = ne_summary.deme.map(month)


In [96]:
ne_pred = ne_summary.iloc[2:]
mig_pred = ne_summary.iloc[1:2]

In [97]:
ne_pred

,deme,mean_Ne_log,mean_Ne_linear,median_Ne_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,lower_hpd_linear,predictor
0,_region_300Clock,0.974816,9.436609,0.916874,1.741570,0.345521,1.061502,0.605930,55.153080,2.215750,Clock
0,region_cases_weekly_prevalence,1.811690,64.817149,1.809091,2.333381,1.293268,1.998874,1.643755,215.467098,19.645716,Weekly Prevalence
0,month_predictor_0,-1.090373,0.081213,-1.105467,0.000000,-1.890191,-0.836518,-1.453160,1.000000,0.012877,April 2022
0,month_predictor_1,1.349364,22.354456,1.327705,2.295658,0.448627,1.591076,0.974446,197.541130,2.809489,May 2022
0,month_predictor_2,0.879787,7.582063,0.903319,1.605495,0.000000,1.217403,0.630435,40.317611,1.000000,June 2022
0,month_predictor_3,0.350134,2.239410,0.265768,1.271516,-0.359538,0.447114,-0.001437,18.685979,0.436981,July 2022
0,month_predictor_4,-0.218935,0.604039,0.000000,0.551484,-1.181620,0.000518,-0.337461,3.560283,0.065823,August 2022
0,month_predictor_5,-0.279185,0.525793,-0.106759,0.612273,-1.455882,0.000133,-0.474297,4.095176,0.035004,September 2022
0,month_predictor_6,-0.517517,0.303727,-0.511033,0.390065,-1.593811,0.000000,-0.659828,2.455079,0.025479,October 2022
0,month_predictor_7,0.144945,1.396191,0.000000,1.754991,-1.188099,0.369089,-0.240939,56.884106,0.064849,November 2022


In [118]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(ne_pred).mark_point(filled=True, color='black', width = 5).encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False, labelFontSize= 24,titleFontSize= 24) ),
  x=alt.X('predictor', title = "Predictor", sort= list(month.values()),  axis=alt.Axis(labelFontSize= 24,titleFontSize= 24,labelLimit = 0, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" or datum.predictor != "air").properties(
    width=850,
    height=300
)

hpd = alt.Chart(ne_pred).mark_rule(strokeWidth = 2).encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("predictor",sort =  list(month.values()))
).transform_filter(datum.predictor != "Clock").properties(
    width=850,
    height=300
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=850,
    height=300
)

predict_coeff = (points + hpd + one_line).properties(title='D')
predict_coeff.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 35
).configure_axis(
    labelFontSize=60,
    titleFontSize=60
).configure_legend(
    labelFontSize = 60)




alt.LayerChart(...)

In [105]:
# error_bars = alt.Chart(mig_short).mark_errorbar(extent='ci').encode(
#   x=alt.X('mig_per_bl:Q', scale=alt.Scale(zero=False)),
#   y=alt.Y('migration_direction:N')
# )

points = alt.Chart(mig_pred).mark_point(filled=True, color='black').encode(
  y=alt.Y('mean_Ne_log:Q',axis = alt.Axis(title = "Coefficient", grid =False, labelFontSize= 18,titleFontSize= 16) ),
  x=alt.X('predictor', title = "Predictor",  axis=alt.Axis(labelFontSize= 16,titleFontSize= 16, labelAngle=-35)),
).transform_filter(datum.predictor != "Clock" ).properties(
    width=200,
    height=200
)

hpd = alt.Chart(mig_pred).mark_rule().encode(
    y2=alt.Y2("upper_hpd_log_95"),
    y = alt.Y("lower_hpd_log_95"),
    x=alt.X("predictor")
).transform_filter(datum.predictor != "Clock").properties(
    width=300,
    height=150
)

one_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=300,
    height=150
)

predict_coeff = (points + hpd ).properties(title='B')
predict_coeff.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start', fontSize= 25
).configure_axis(
    labelFontSize=24,
    titleFontSize=24
).configure_legend(
    labelFontSize = 60)




alt.LayerChart(...)